In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import socket
from itertools import product
import numpy as np
import json


In [2]:
X,y = fetch_california_housing(return_X_y=True)

In [3]:
fetch_california_housing()

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n--------------------

In [4]:
y

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [7]:
params = [
        {
        'model':'xgboost',
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': range(1, 11, 1),
#         'n_estimators':range(50, 400, 25),
        },
#         {'model':'lightgbm',
#          'num_leaves': range(10, 110, 50),
#          'min_child_samples':range(100, 1250, 250),
#          'max_bin':range(3, 18, 3),
#           'bagging_fraction': [0.6,0.8, 1,1.2],
#           'max_depth': range(8, 17, 2),
#           'min_split_gain': [0.001, 0.1],
#           'min_child_weight': [30, 40,50]
#           },
#         {
#           'model':'catboost',
#           'depth':range(1, 4, 2),
#           'learning_rate':[0.03,0.001,0.01], 
#           'iterations':[100],
#           'l2_leaf_reg':[3,1,5,10,100],
#           'border_count':[32,5,10,20,50,100,200],
#           'ctr_border_count':[50,5,10,20,100,200],
#           'thread_count':[1],
#         },
]



def split_params(jobs_param,chunk):
    li = []
    for i,x in enumerate(jobs_param):
        model = x.pop('model')
        c_plane = [dict(zip(x, v)) for v in product(*x.values())]
        for y in c_plane:
            y.update( {"model":model})
        li+=c_plane
    split_list=np.array_split(li, chunk)
    json_list = [json.dumps(i.tolist()) for i in split_list]
    print(f"NumberOfParams = {len(li)}, ChunkSize = {len(split_list[0])}, NumberOfJobs = {chunk}")
    return json_list

split = split_params(params,2)


NumberOfParams = 15, ChunkSize = 8, NumberOfJobs = 2


In [8]:
split

['[{"min_child_weight": 1, "gamma": 0.5, "model": "xgboost"}, {"min_child_weight": 1, "gamma": 1, "model": "xgboost"}, {"min_child_weight": 1, "gamma": 1.5, "model": "xgboost"}, {"min_child_weight": 1, "gamma": 2, "model": "xgboost"}, {"min_child_weight": 1, "gamma": 5, "model": "xgboost"}, {"min_child_weight": 5, "gamma": 0.5, "model": "xgboost"}, {"min_child_weight": 5, "gamma": 1, "model": "xgboost"}, {"min_child_weight": 5, "gamma": 1.5, "model": "xgboost"}]',
 '[{"min_child_weight": 5, "gamma": 2, "model": "xgboost"}, {"min_child_weight": 5, "gamma": 5, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 0.5, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 1, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 1.5, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 2, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 5, "model": "xgboost"}]']

In [11]:
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters(host="localhost"))
channel = connection.channel()

channel.queue_declare(queue="task_queue", durable=True)

for msg in split:
    channel.basic_publish(
        exchange="",
        routing_key="task_queue",
        body=msg,
        properties=pika.BasicProperties(delivery_mode=2),  # make message persistent
    )
    print(" [x] Sent %r" % msg)

connection.close()

 [x] Sent '[{"min_child_weight": 1, "gamma": 0.5, "model": "xgboost"}, {"min_child_weight": 1, "gamma": 1, "model": "xgboost"}, {"min_child_weight": 1, "gamma": 1.5, "model": "xgboost"}, {"min_child_weight": 1, "gamma": 2, "model": "xgboost"}, {"min_child_weight": 1, "gamma": 5, "model": "xgboost"}, {"min_child_weight": 5, "gamma": 0.5, "model": "xgboost"}, {"min_child_weight": 5, "gamma": 1, "model": "xgboost"}, {"min_child_weight": 5, "gamma": 1.5, "model": "xgboost"}]'
 [x] Sent '[{"min_child_weight": 5, "gamma": 2, "model": "xgboost"}, {"min_child_weight": 5, "gamma": 5, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 0.5, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 1, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 1.5, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 2, "model": "xgboost"}, {"min_child_weight": 10, "gamma": 5, "model": "xgboost"}]'
